# corona 데이터 로드 
1. 데이터 로드 
2. unnamed : 0, seq, statDt, statetime, updateDt 컬럼 삭제
3. 인덱스 값을 creat Dt로 변경 (시계열로 변환하여 인덱스로 이동)
4. 컬럼의 이름을 변경 : 사망자 확진자 누적의심자 누적확진율
5. 인덱스의 이름 등록일시로 변경
6. 일일확진자 일일사망자 컬럼 추가 사망자, 확진자의 차이 대입 결측치 0으로 대체
7. 누적 확진자와 누적 확진률 결측치 데이터는 전의 데이터 값으로 대체
8. 인덱스를 기준으로 오름차순 변경 
9. 일일 확진자가 음수인 값을 찾아서 인덱스 삭제

In [102]:
import pandas as pd

In [103]:
df = pd.read_csv("../csv/corona.csv", usecols=[1,2,3,8,9],parse_dates=[0], index_col= 0)
df.head(2)

,deathCnt,decideCnt,accExamCnt,accDefRate
createDt,,,,
2022-06-08 09:09:05.982,24305,18188200,NaN,NaN
2022-06-07 09:09:00.897,24299,18174842,NaN,NaN


In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 820 entries, 2022-06-08 09:09:05.982000 to 2020-03-10 00:00:00
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   deathCnt    820 non-null    int64  
 1   decideCnt   820 non-null    int64  
 2   accExamCnt  692 non-null    float64
 3   accDefRate  623 non-null    float64
dtypes: float64(2), int64(2)
memory usage: 32.0 KB


In [105]:
df.columns = ["사망자", "확진자", "누적의심자","누적확진율"]
df.index.name = "등록일시"
df.head(1)

,사망자,확진자,누적의심자,누적확진율
등록일시,,,,
2022-06-08 09:09:05.982,24305,18188200,NaN,NaN


In [106]:
df = df.fillna(method="bfill")
df.head(1)

,사망자,확진자,누적의심자,누적확진율
등록일시,,,,
2022-06-08 09:09:05.982,24305,18188200,21518073.0,2.787908


In [107]:
df["일일사망자"] = (df["사망자"]- df["사망자"].shift(-1)).fillna(0)
df["일일확진자"] = df["확진자"].diff(-1).fillna(0)
df.head()

,사망자,확진자,누적의심자,누적확진율,일일사망자,일일확진자
등록일시,,,,,,
2022-06-08 09:09:05.982,24305,18188200,21518073.0,2.787908,6.0,13358.0
2022-06-07 09:09:00.897,24299,18174842,21518073.0,2.787908,20.0,6172.0
2022-06-06 09:00:06.734,24279,18168670,21518073.0,2.787908,21.0,5022.0
2022-06-05 08:53:19.426,24258,18163648,21518073.0,2.787908,20.0,9834.0
2022-06-04 08:56:49.219,24238,18153814,21518073.0,2.787908,9.0,12039.0


In [110]:
df.head()

,사망자,확진자,누적의심자,누적확진율,일일사망자,일일확진자
등록일시,,,,,,
2020-03-10,54,7513,210144.0,3.919308,0.0,0.0
2020-03-11,60,7755,222395.0,3.804175,6.0,242.0
2020-03-12,66,7869,234998.0,3.621744,6.0,114.0
2020-03-13,67,7979,248647.0,3.458499,1.0,110.0
2020-03-14,72,8086,261335.0,3.318000,5.0,107.0


In [108]:
df = df.sort_index()
df


,사망자,확진자,누적의심자,누적확진율,일일사망자,일일확진자
등록일시,,,,,,
2020-03-10 00:00:00.000,54,7513,210144.0,3.919308,0.0,0.0
2020-03-11 00:00:00.000,60,7755,222395.0,3.804175,6.0,242.0
2020-03-12 00:00:00.000,66,7869,234998.0,3.621744,6.0,114.0
2020-03-13 00:00:00.000,67,7979,248647.0,3.458499,1.0,110.0
2020-03-14 00:00:00.000,72,8086,261335.0,3.318000,5.0,107.0
...,...,...,...,...,...,...
2022-06-04 08:56:49.219,24238,18153814,21518073.0,2.787908,9.0,12039.0
2022-06-05 08:53:19.426,24258,18163648,21518073.0,2.787908,20.0,9834.0
2022-06-06 09:00:06.734,24279,18168670,21518073.0,2.787908,21.0,5022.0


In [114]:
df[df["일일확진자"] < 0]

,사망자,확진자,누적의심자,누적확진율,일일사망자,일일확진자
등록일시,,,,,,
